# Найдем самые лучшие из степенных факторов используя модель линейной регрессии

In [1]:
import optuna
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression, ElasticNet
from sklearn.isotonic import IsotonicRegression
from sklearn.metrics import mean_squared_log_error, mean_squared_error
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv('energy_2.csv')
print(data.head())

             timestamp  meter_reading  air_temperature  cloud_coverage  \
0  2016-01-30 08:00:00        43.6839              8.3             0.0   
1  2016-01-31 05:00:00        37.5408             12.8             0.0   
2  2016-01-31 17:00:00        52.5571             20.6             0.0   
3  2016-04-08 14:00:00        59.3827             21.7             2.0   
4  2016-05-01 19:00:00       448.0000             31.1             0.0   

   dew_temperature  precip_depth_1_hr  sea_level_pressure  wind_speed  \
0              6.1                0.0              1019.0         2.1   
1             10.0                0.0              1021.9         0.0   
2             11.7                0.0              1020.9         1.5   
3             14.4                0.0              1015.1         3.1   
4             17.2                0.0              1016.1         4.1   

   wind_direction_sin  wind_direction_cos  air_temperature1  hour  
0           -0.642788           -0.766044       

# Подбор параметров

In [3]:
def calculate_bic(y, y_pred, power):
    return len(y)*np.log(len(y)*mean_squared_error(y, y_pred)**2) + power*np.log(len(y))

In [4]:
columns=list(data.columns)
columns.remove('timestamp')
columns.remove('meter_reading')
columns.remove('hour')

# Линейные значения

In [20]:
best_columns = []
current_columns = []
power = 0
best_power = 0
bic_best = 10000000
for column in columns:
    current_columns.append(column)
    power += 1
    y = data['meter_reading']
    x = MinMaxScaler().fit_transform(data[current_columns])
    bic= calculate_bic(y, LinearRegression().fit(x, y).predict(x), power)
    if bic < bic_best:
        best_columns.append(column)
        best_power += 1
        bic_best = bic
    else:
        power -= 1
        current_columns.remove(column)
print(best_columns)        
    

['air_temperature', 'dew_temperature', 'precip_depth_1_hr', 'sea_level_pressure', 'wind_speed', 'air_temperature1']


# Квадратичное значение

In [6]:
for column in columns:
    data[column +'_2'] = np.multiply(data[column], data[column])

In [24]:
for column in columns:
    c = column + '_2'
    current_columns.append(c)
    power += 1
    y = data['meter_reading']
    x = MinMaxScaler().fit_transform(data[current_columns])
    bic= calculate_bic(y, LinearRegression().fit(x, y).predict(x), power)
    if bic < bic_best:
        best_columns.append(c)
        best_power += 1
        bic_best = bic
    else:
        power -= 1
        current_columns.remove(c)
print(best_columns)        
    

['air_temperature', 'dew_temperature', 'precip_depth_1_hr', 'sea_level_pressure', 'wind_speed', 'air_temperature1', 'air_temperature_2', 'cloud_coverage_2', 'dew_temperature_2', 'sea_level_pressure_2', 'wind_speed_2']


# Кубические значения

In [25]:
for column in columns:
    data [column + '_3']=np.multiply(data[column + '_2'], data[column])

In [27]:
for column in columns:
    c = column + '_3'
    current_columns.append(c)
    power += 1
    y = data['meter_reading']
    x = MinMaxScaler().fit_transform(data[current_columns])
    bic= calculate_bic(y, LinearRegression().fit(x, y).predict(x), power)
    if bic < bic_best:
        best_columns.append(c)
        best_power += 1
        bic_best = bic
    else:
        power -= 1
        current_columns.remove(c)
print(best_columns)   

['air_temperature', 'dew_temperature', 'precip_depth_1_hr', 'sea_level_pressure', 'wind_speed', 'air_temperature1', 'air_temperature_2', 'cloud_coverage_2', 'dew_temperature_2', 'sea_level_pressure_2', 'wind_speed_2', 'air_temperature_3', 'cloud_coverage_3', 'sea_level_pressure_3', 'wind_direction_sin_3', 'wind_direction_cos_3']


In [28]:
data_norm = pd.DataFrame(MinMaxScaler().fit_transform(data[best_columns]))

# Разделение данных  на обучающие и проверочные модели

In [29]:
train, test, y_train, y_test = train_test_split(data_norm, data['meter_reading'], test_size=0.2)

# Модели регрессии
Линейной и изитерической регрессии

In [12]:
def rmsle_err(y, y_pred):
    return((np.log(1+y)-np.log(1+y_pred))**2).mean()**0.5

In [13]:
y = data['meter_reading']
model1 = LinearRegression().fit(train, y_train)
print('RMSLE:{0:.5}'.format(rmsle_err(y_train, model1.predict(train))))

RMSLE:0.19566


In [14]:
model2 = IsotonicRegression(out_of_bounds='clip').fit(train[0], y_train)
print('RMSLE:{0:.5}'.format(rmsle_err(y_train, model2.predict(train[0]))))

RMSLE:0.21745


# ElasticNet
Выберем наилучшие гиперпараметры модели

In [31]:
def objective_elastic(trial):
    alpha = trial.suggest_float('alpha', 1e-8, 1, log=True)
    l1_ratio = trial.suggest_float('l1_ratio', 1e-3, 1, log=True)
    regressor_obj = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, max_iter = 10000, tol = 1)
    regressor_obj.fit(train, y_train)
    y_pred= regressor_obj.predict(train)
    return mean_squared_log_error(y_train, y_pred)

In [32]:
study_elastic = optuna.create_study()
study_elastic.optimize(objective_elastic, n_trials=100)

[I 2022-06-19 09:42:49,641] A new study created in memory with name: no-name-af542709-6269-4c0b-9def-0d958034d58c
[I 2022-06-19 09:42:49,855] Trial 0 finished with value: 0.04266637762192197 and parameters: {'alpha': 0.0009732111743282171, 'l1_ratio': 0.40602640279871466}. Best is trial 0 with value: 0.04266637762192197.
[I 2022-06-19 09:42:49,870] Trial 1 finished with value: 0.042704963998789036 and parameters: {'alpha': 0.0002699826999467476, 'l1_ratio': 0.02403653333965759}. Best is trial 0 with value: 0.04266637762192197.
[I 2022-06-19 09:42:49,882] Trial 2 finished with value: 0.04273784639424791 and parameters: {'alpha': 1.4810342036706705e-07, 'l1_ratio': 0.001916048913192047}. Best is trial 0 with value: 0.04266637762192197.
[I 2022-06-19 09:42:49,892] Trial 3 finished with value: 0.04273785814178568 and parameters: {'alpha': 5.5972652470358775e-08, 'l1_ratio': 0.016221553352145042}. Best is trial 0 with value: 0.04266637762192197.
[I 2022-06-19 09:42:49,916] Trial 4 finished 

In [40]:
model3 = ElasticNet(alpha=study_elastic.best_params['alpha'],
                   l1_ratio=study_elastic.best_params['l1_ratio'],
                   max_iter = 100000, tol=0.01).fit(train, y_train)
print('RMSLE:{0:.5}'.format(rmsle_err(y_train, model3.predict(train))))

RMSLE:0.20281


# Объедиение моделей

# Используем оптуна для поиска оптимального коэффециента

In [44]:
def objective(trial):
    alpha = trial.suggest_float('alpha', 1e-10, 1, log=True)
    beta = trial.suggest_float('beta', 1e-10, 1-alpha, log=True)
    y_pred= (alpha*model1.predict(test)+
            beta*model2.predict(test[0])+
            (1-alpha-beta)*model3.predict(test))
    return mean_squared_log_error(y_test, y_pred)

In [45]:
study = optuna.create_study()
study.optimize(objective,n_trials=100)

[I 2022-06-19 10:28:15,157] A new study created in memory with name: no-name-1f082ae1-ac27-4856-b926-4d7f5b44468e
[I 2022-06-19 10:28:15,174] Trial 0 finished with value: 0.04952701227962745 and parameters: {'alpha': 7.896163493507556e-05, 'beta': 1.0797980427792678e-10}. Best is trial 0 with value: 0.04952701227962745.
[I 2022-06-19 10:28:15,188] Trial 1 finished with value: 0.04758981945989846 and parameters: {'alpha': 0.2345985927840193, 'beta': 0.0027515402106223033}. Best is trial 1 with value: 0.04758981945989846.
[I 2022-06-19 10:28:15,205] Trial 2 finished with value: 0.04948828410327212 and parameters: {'alpha': 0.004343120313333712, 'beta': 5.328927776028681e-08}. Best is trial 1 with value: 0.04758981945989846.
[I 2022-06-19 10:28:15,223] Trial 3 finished with value: 0.049527443211617894 and parameters: {'alpha': 5.449608524805599e-10, 'beta': 0.00011351683064474555}. Best is trial 1 with value: 0.04758981945989846.
[I 2022-06-19 10:28:15,249] Trial 4 finished with value: 0.

In [46]:
print(study.best_params)

{'alpha': 0.9921226833180047, 'beta': 0.007317691582675192}


In [49]:
y_pred1 = model1.predict(data_norm)
y_pred2 = model2.predict(data_norm[0])
y_pred3 = model3.predict(data_norm)
y_pred = (study.best_params['alpha']*y_pred1+
          study.best_params['beta']*y_pred2+
          (1-study.best_params['alpha']-study.best_params['beta'])*y_pred3)
print('RMSLE линейной регрессии:{0:.5}'.format(rmsle_err(y, y_pred1)))
print('RMSLE изотенической регрессии:{0:.5}'.format(rmsle_err(y, y_pred2)))
print('RMSLE ElasticNet:{0:.5}'.format(rmsle_err(y, y_pred3)))
print('RMSLE ансамбля:{0:.5}'.format(rmsle_err(y, y_pred)))

RMSLE линейной регрессии:0.19453
RMSLE изотенической регрессии:0.21608
RMSLE ElasticNet:0.20691
RMSLE ансамбля:0.19456
